In [11]:
from utilities import utils

In [2]:
import pandas as pd
from sklearn import preprocessing
from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [14]:
util = utils.Utils()
data_dir = util.data_dir
print(data_dir)
image_dir = data_dir + '/' + 'all_classes'
print(image_dir)

/Users/yachnasharma/Downloads/case_study_visual_intelligence/VIntel/data/home_depot_products
/Users/yachnasharma/Downloads/case_study_visual_intelligence/VIntel/data/home_depot_products/all_classes


In [6]:
data = pd.read_csv('data.csv')

In [17]:
data.sample(5)
columns_to_keep = ['image_name', 'bar_stool', 'bookcase',
                  'chandelier', 'dining_chair', 'market_umbrella',
                  'night_stands', 'ottoman', 'sconces', 'table_lamp',
                  'vases']
cols_to_drop = [c for c in data.columns if c not in columns_to_keep]
data = data.drop(cols_to_drop, axis=1)
data['Filenames'] = data['image_name']
data = data.drop(['image_name'], axis=1)

In [18]:
data.sample(5)

,bar_stool,bookcase,chandelier,dining_chair,market_umbrella,night_stands,ottoman,sconces,table_lamp,vases,Filenames
2740,0,0,0,0,0,0,0,0,1,0,d0296fae-44d0-4e86-8f43-6ea32b838f8c.jpg
2609,0,0,0,0,0,0,0,0,1,0,32bdbbc8-27a8-440c-8a80-5b9f018065d0.jpg
981,0,0,1,0,0,0,0,0,0,0,b663414f-a648-4b4a-8cb2-c2e492ff3c70.jpg
2109,0,0,0,0,0,0,1,0,0,0,3bf069af-1ff3-4ccd-9ead-df007eb53b8b.jpg
73,1,0,0,0,0,0,0,0,0,0,92fc5d99-2372-44d9-96d9-0aa231bc78c0.jpg


In [22]:
columns = [c for c in columns_to_keep if c != 'image_name']
print(columns)

['bar_stool', 'bookcase', 'chandelier', 'dining_chair', 'market_umbrella', 'night_stands', 'ottoman', 'sconces', 'table_lamp', 'vases']


In [20]:
data.shape

(2964, 11)

In [21]:
df = data

In [41]:
df = df.sample(frac=1).reset_index(drop=True)

In [51]:
df[2500:].shape

(464, 11)

In [46]:
datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)
train_generator = datagen.flow_from_dataframe(
dataframe = df[:1500],
directory = image_dir,
x_col = "Filenames",
y_col = columns,
batch_size = 32,
seed = 42,
shuffle = True,
class_mode = "other",
target_size = (256,256))

valid_generator = test_datagen.flow_from_dataframe(
dataframe = df[1500:2500],
directory = data_dir,
x_col = "Filenames",
y_col = columns,
batch_size = 32,
seed = 42,
shuffle = True,
class_mode = "other",
target_size = (256,256))

test_generator = test_datagen.flow_from_dataframe(
dataframe = df[2500:],
directory = data_dir,
x_col = "Filenames",
batch_size = 16,
seed = 42,
shuffle = True,
class_mode = None,
target_size = (256,256))

Found 1500 validated image filenames.
Found 0 validated image filenames.
Found 0 validated image filenames.


In [47]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [48]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
print(valid_generator.n)
# print(STEP_SIZE_VALID)
# model.fit_generator(generator = train_generator,
#                     steps_per_epoch = STEP_SIZE_TRAIN,
#                     validation_data = valid_generator,
#                     validation_steps = STEP_SIZE_VALID,
#                     epochs = 10
# )

0
